# Prediction Insights

## Overview

Howso Engine enables powerful predictions with complete attribution and detailed explanations to make learning from
and debugging your data and predictions as easy as possible. For more information on predictions with Howso Engine,
check out the [predictions user guide](https://docs.howso.com/user_guide/basic_capabilities/predictions.html).

In [1]:
from pprint import pprint

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Setup

This recipe will focus on the insights gained from predictions. Specifically, insights from:

- Influential Cases,
- Feature Contributions,
- Residual and Similarity Conviction,
- and Categorical Action Probabilities.

The [basic workflow guide](https://docs.howso.com/user_guide/basic_capabilities/basic_workflow.html) covers creating a Trainee and performing a basic react in detail.

### Load Data and Create Trainee

In [2]:
df = pd.read_csv("../../data/iris/iris.tsv.gz", sep="\t", compression="gzip")
train_data = df.iloc[:-30]
new_data = df[~df.index.isin(train_data.index)]
features = infer_feature_attributes(train_data)

t = Trainee(features=features)

df

,Unnamed: 0,sepal length,sepal width,petal length,petal width,class
0,27,5.2,3.5,1.5,0.2,Iris-setosa
1,14,5.8,4.0,1.2,0.2,Iris-setosa
2,25,5.0,3.0,1.6,0.2,Iris-setosa
3,82,5.8,2.7,3.9,1.2,Iris-versicolor
4,97,6.2,2.9,4.3,1.3,Iris-versicolor
...,...,...,...,...,...,...
145,126,6.2,2.8,4.8,1.8,Iris-virginica
146,124,6.7,3.3,5.7,2.1,Iris-virginica
147,78,6.0,2.9,4.5,1.5,Iris-versicolor
148,125,7.2,3.2,6.0,1.8,Iris-virginica


### Train, Analyze, and React

In [3]:
t.train(train_data)
t.analyze()
t.react_into_features(similarity_conviction=True)

reaction = t.react(
    contexts=new_data,
    context_features=["sepal length", "sepal width", "petal length", "petal width"],
    action_features=["class"],
    details={
        "influential_cases": True,
        "similarity_conviction": True,
        "feature_contributions_robust": True,
        "feature_residuals_robust": True,
        "local_case_feature_residual_convictions_robust": True,
        "categorical_action_probabilities": True,
    }
)

Note that, unlike in the basic workflow guide, we include several `details` in the react call and predict multiple cases as opposed to one. These are what will enable the
insights that we're going to get after the predictions are made.

For more information, see [the API documentation for `Trainee.react()`](https://docs.howso.com/api_reference/_autosummary/howso.engine.html#howso.engine.Trainee.react).

### Inspect the Predictions

Howso Engine has high accuracy even on small datasets.

Here we see a comparison of the trained data (dots) and the predicted data (stars). This plot shows that the predictions align well with the trained data.

In [4]:
train_data = train_data.astype({"class": str})
predicted_data = pd.concat([new_data.reset_index(drop=True).drop(columns="class"), reaction["action"]], axis=1)
predicted_data = predicted_data.astype({"class": str})

cmap = px.colors.qualitative.D3
fig = go.Figure()
for i, (label, group) in enumerate(train_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        name=label,
        marker=dict(color=cmap[i], opacity=0.75),
        legendgroup="trained",
        legendgrouptitle_text="Trained class",
    ))

for i, (label, group) in enumerate(predicted_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        marker=dict(size=12, symbol="star", color=cmap[i], opacity=0.75),
        name=label,
        legendgroup="predicted",
        legendgrouptitle_text="Predicted class",
        hovertext=group.index,
    ))

fig.update_layout(
    xaxis_title="Petal Length",
    yaxis_title="Petal Width",
    width=1250,
    title="Trained and Predicted Values"
)
fig.show()

For categorical action features, the prediction can be further understood with the `categorical_action_probabilities` detail.  
This shows the probability of each possible value of the target feature. This information can highlight cases that are on the
border of two classes, like some of the above points are.  The closer a case gets to a class border, the more mixed the
categorical action probabilities may get.

In [5]:
pprint(reaction["details"]["categorical_action_probabilities"], compact=True)

[{'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-versicolor': 0.2595332698293332,
            'Iris-virginica': 0.7404667301706668}},
 {'class': {'Iris-versicolor': 0.10794626774761718,
            'Iris-virginica': 0.8920537322523828}},
 {'class': {'Iris-versicolor': 0.22564354170984102,
            'Iris-virginica': 0.774356458290159}},
 {'class': {'Iris-versicolor': 0.5286933842008528,
            'Iris-virginica': 0.47130661579914723}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.1589391970055096,
            'Iris-virginica': 0.8410608029944904}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-virginica': 1}}, {'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.28037551436519886,
            'Iris-virginica': 0.7196244856348012}},
 {'class': {'Iris-versicolor': 0.9324953588068614,
            'Iris-virginica': 0.06750464119313854}},

For example, if we look at case `4`, we see a mix of probabilities because this case is on the border of two classes.

In [6]:
reaction["details"]["categorical_action_probabilities"][4]

{'class': {'Iris-virginica': 0.47130661579914723,
  'Iris-versicolor': 0.5286933842008528}}

### Insight 1: Which Cases Contributed?

Howso provides complete attribution for any and all predictions, showing exactly which cases influenced each prediction.  This can be used to understand or debug predictions.  For instance,
we can inspect the influential cases for one of the cases (case `4`) that was on the decision boundary in the plot above.  We can see that there is a mix of class values and no cases stand
out in terms of influence weight:

In [7]:
inf_cases_4 = pd.DataFrame(reaction["details"]["influential_cases"][4])
inf_cases_4

,.session_training_index,petal length,class,.session,sepal length,.influence_weight,petal width,sepal width
0,89,4.7,Iris-versicolor,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,6.1,0.131990,1.4,2.9
1,26,4.6,Iris-versicolor,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,6.1,0.110172,1.4,3.0
2,61,5.6,Iris-virginica,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,6.3,0.087555,1.8,2.9
3,100,4.7,Iris-versicolor,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,6.1,0.076773,1.2,2.8
4,107,4.9,Iris-versicolor,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,6.3,0.076572,1.5,2.5
5,99,5.1,Iris-virginica,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,6.3,0.074906,1.5,2.8
6,36,5.6,Iris-virginica,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,6.4,0.068019,2.1,2.8
7,30,4.0,Iris-versicolor,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,6.1,0.067561,1.3,2.8
8,57,4.8,Iris-versicolor,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,6.8,0.065627,1.4,2.8
9,34,5.0,Iris-virginica,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,6.0,0.065129,1.5,2.2


Compare that to the influential cases for case `20`, which is firmly in the center of the solo cluster.  Here we see a single class value.

In [8]:
inf_cases_20 = pd.DataFrame(reaction["details"]["influential_cases"][20])
inf_cases_20

,.session_training_index,petal length,class,.session,sepal length,.influence_weight,petal width,sepal width
0,63,1.6,Iris-setosa,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,5.0,0.136586,0.4,3.4
1,9,1.7,Iris-setosa,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,5.4,0.109489,0.2,3.4
2,85,1.5,Iris-setosa,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,5.1,0.091502,0.2,3.4
3,117,1.5,Iris-setosa,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,5.1,0.087351,0.4,3.7
4,69,1.5,Iris-setosa,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,5.0,0.079467,0.2,3.4
5,104,1.5,Iris-setosa,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,5.4,0.078586,0.2,3.7
6,47,1.7,Iris-setosa,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,5.4,0.075887,0.4,3.9
7,21,1.3,Iris-setosa,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,5.4,0.075887,0.4,3.9
8,43,1.4,Iris-setosa,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,5.2,0.064990,0.2,3.4
9,0,1.5,Iris-setosa,b18c96d3-ce53-4633-a7fe-4bc59e9c83f4,5.2,0.060322,0.2,3.5


With the influential cases we can derive additional insights, such as identifying anomalous cases from within the influential cases using `similarity_conviction`.  
This can help to identify data that are making predictions more noisy or identify what type of data should be collected to improve predictive power in the future.

In [9]:
inf_case_indices = inf_cases_4[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"]
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
9,6.0,2.2,5.0,1.5,Iris-virginica,0.539253
4,6.3,2.5,4.9,1.5,Iris-versicolor,0.792240
5,6.3,2.8,5.1,1.5,Iris-virginica,0.898850
8,6.8,2.8,4.8,1.4,Iris-versicolor,0.979947
11,6.4,2.7,5.3,1.9,Iris-virginica,1.053570
1,6.1,3.0,4.6,1.4,Iris-versicolor,1.061901
3,6.1,2.8,4.7,1.2,Iris-versicolor,1.098104
2,6.3,2.9,5.6,1.8,Iris-virginica,1.116685
12,6.1,3.0,4.9,1.8,Iris-virginica,1.157880
7,6.1,2.8,4.0,1.3,Iris-versicolor,1.168627


In [10]:
inf_case_indices = inf_cases_20[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"],
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
6,5.4,3.9,1.7,0.4,Iris-setosa,0.788957
7,5.4,3.9,1.3,0.4,Iris-setosa,0.862877
11,4.6,3.4,1.4,0.3,Iris-setosa,0.909134
10,5.3,3.7,1.5,0.2,Iris-setosa,0.924620
0,5.0,3.4,1.6,0.4,Iris-setosa,0.966207
1,5.4,3.4,1.7,0.2,Iris-setosa,1.058589
3,5.1,3.7,1.5,0.4,Iris-setosa,1.096762
5,5.4,3.7,1.5,0.2,Iris-setosa,1.127433
2,5.1,3.4,1.5,0.2,Iris-setosa,1.259184
12,5.1,3.5,1.4,0.3,Iris-setosa,1.334616


### Insight 2: Which Features Contributed?

In addition to providing attribution to cases, Howso also provides robust feature contributions to explain which features contributed to each prediction.
When inspecting the feature importances for case `13`,  we see that `petal width` and `petal length` provided the majority of the contribution to the 
prediction, whereas for case `20` `petal width` and `petal length` only have slightly higher contributions than the other features.  This could indicate 
that different features are more important for predicting different classes within this dataset and that focusing on those features would be prudent.

Identifying that certain cases that influenced a prediction are anomalous can help to explain low predictive power in certain parts of the data or other
issues that may appear during deployment.

In [11]:
fcs_13 = pd.DataFrame(reaction["details"]["feature_contributions_robust"][13:14], index=[13])
fcs_20 = pd.DataFrame(reaction["details"]["feature_contributions_robust"][20:21], index=[20])

display(fcs_13)
display(fcs_20)

,petal length,sepal length,petal width,sepal width
13,0.309364,0.134326,0.318782,0.06568


,petal length,sepal length,petal width,sepal width
20,0.160318,0.117579,0.159647,0.122506


### Insight 3: How Certain is the Prediction?

[Residuals](https://docs.howso.com/user_guide/basic_capabilities/residuals.html) can characterize the uncertainty of the data around the prediction.  This will tell us which features are hard to predict in the region of the data around each case we're predicting.
If a prediction is less accurate than expected, this can explain which features were noisy and may have contributed to the problem.

In [12]:
pd.DataFrame(reaction["details"]["feature_residuals_robust"])

,petal length,class,sepal length,petal width,sepal width
0,0.316345,0.246918,0.419110,0.155347,0.200967
1,0.455092,0.361243,0.379007,0.273793,0.236921
2,0.499980,0.223748,0.429601,0.326543,0.154511
3,0.417547,0.349564,0.372912,0.259437,0.217842
4,0.410694,0.255052,0.352748,0.203288,0.266140
5,0.336986,0.128101,0.360771,0.134115,0.157232
6,0.428619,0.096348,0.317292,0.206709,0.213859
7,0.462920,0.288807,0.385860,0.301305,0.154439
8,0.337946,0.214845,0.333649,0.154813,0.233382
9,0.355052,0.119362,0.333177,0.140930,0.171296


We can also use [`residual conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#prediction-residual-conviction) to determine which features
are uncertain in a scale-invariant manner, which can be useful if you wish to compare different features of different scales against each other.  Higher residual conviction
indicates something that is less surprising than expected and lower redsidual conviction indicates the opposite.

In [13]:
pd.DataFrame(reaction["details"]["local_case_feature_residual_convictions_robust"])

,petal length,class,sepal length,petal width,sepal width
0,0.955049,1.066269,1.175051,0.802160,1.746456
1,1.377457,1.032291,1.570590,1.180661,0.663548
2,1.062967,1.520404,1.049181,0.983285,2.624776
3,0.887317,1.112053,0.699126,1.121109,0.490977
4,0.333112,0.723975,1.590878,0.902690,0.721471
5,1.713707,1.407985,1.233485,1.390252,0.807158
6,0.613218,0.587237,1.014973,0.881732,0.663201
7,1.159223,1.029703,1.041573,1.990364,0.710780
8,0.804377,1.008368,0.611964,1.015961,0.343099
9,1.444208,1.101478,1.172914,1.009093,0.563643


### Insight 4: How Anomalous are the Predicted Cases?

By getting the [`similarity conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#similarity-conviction) of the cases that we predict, 
we can determine which of them are anomalous relative to the trained data. This could help to highlight cases that are unusually difficult or easy to predict, such as can be the case
when model drift occurs as a form of model monitoring.

In [14]:
pprint(reaction["details"]["similarity_conviction"], compact=True)

[2.458163722403125, 1.5355268470498389, 1.9703396848065213, 1.1932277418499297,
 0.9049533623206016, 2.645340876758963, 2.3189375519914277, 1.763208402071557,
 1.1949532827544833, 2.4724740467619464, 3.3618207743699844, 2.9229072913854584,
 4.439893485653456, 1.6873893824957287, 1.6869596359841073, 1.9149593505246532,
 1.8494864087341076, 2.971890318003325, 1.0423879532581057, 0.8457603752914653,
 1.8788580971742515, 2.291951521601636, 1.0552822155388273, 2.9023506943593493,
 3.170341992118255, 2.0843045143278, 2.0054099979530093, 2.3091253939331033,
 1.39514991898709, 1.2404039707993175]


In this case, there are no cases with a low similarity_conviction which indicates that there are no cases that we reacted to which are particularly
anomalous.